In [1]:
from financialtools.utils import get_tickers
import polars as pl
from financialtools.wrappers import read_financial_results



In [2]:
tickers_sector = get_tickers(columns=['ticker', "sector"])
metrics = read_financial_results()[0]


In [3]:
metrics = metrics.melt(
    id_vars=['ticker', 'time'],
    var_name='metrics',
    value_name='value'
)

In [4]:
metrics_sector = (
    pl.from_pandas(
        metrics.merge(tickers_sector.to_pandas(), on='ticker', how='left')
        )
    .group_by(["sector", 'metrics', "time"])
    .agg(pl.col("value").mean().alias("market_value"))
    .sort(["sector", "metrics", 'time'], descending=[False, False, True])
)

In [5]:
metrics_sector

sector,metrics,time,market_value
str,str,i64,f64
"""Commercial Services""","""CurrentRatio""",2024,1.387463
"""Commercial Services""","""CurrentRatio""",2023,1.5153
"""Commercial Services""","""CurrentRatio""",2022,1.74355
"""Commercial Services""","""CurrentRatio""",2021,1.6717
"""Commercial Services""","""CurrentRatio""",2020,1.1447
…,…,…,…
"""Utilities""","""ROE""",2024,0.132005
"""Utilities""","""ROE""",2023,0.059025
"""Utilities""","""ROE""",2022,0.06997


In [6]:
metrics_sector.to_pandas().to_excel("financial_data/metrics_by_sectors.xlsx", index=False)

In [33]:
eval_metrics = (
    pl.from_pandas(
        read_financial_results()[1]
        .melt(
            id_vars=['ticker', 'time'],
            var_name='eval_metrics',
            value_name='market_value'
        )
        .merge(tickers_sector.to_pandas(), on='ticker', how='left')
        )
    .group_by(["sector", 'eval_metrics', "time"])
    .agg(pl.col("market_value").mean().alias("market_value"))
    .sort(["sector", "eval_metrics", 'time'], descending=[False, False, True])
    .to_pandas()
    )



In [34]:
eval_metrics.to_excel("financial_data/eval_metrics_by_sectors.xlsx", index=False)